cold-user에게 정보를 받는 법 고민해보기

In [1]:
import sys
import os

# 주피터 노트북 환경에서 __file__이 없으므로, 현재 워킹 디렉토리 기준으로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import random
from data_scraping.common.data_loader import load_ratings_data

df_ratings = load_ratings_data()
lot_rated_movie_ids = set(df_ratings.value_counts(subset='movie_id').sort_values(ascending=False).values[:100])

# 여기 있는 영화를 보셨나요?
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
lot_rated_movie_ids = lot_rated_movie_ids - selected_movie_ids

# 버튼 다시 누르면
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))

/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))


# 수집한 거 불러오기

In [2]:
from modeling.utils.data_integration import DataIntegrator

# Firebase 초기화
from user_system.firebase_config import setup_firebase_config
import streamlit as st

# Firebase 설정
firebase_available = setup_firebase_config()
print(f"Firebase 초기화 상태: {firebase_available}")

2025-10-25 21:59:12.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 21:59:12.812 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 21:59:12.863 
  command:

    streamlit run /Users/user/miniconda3/envs/dev/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-25 21:59:12.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 21:59:12.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 21:59:12.864 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new ca

Firebase 초기화 상태: True


In [3]:
# Firebase 초기화 확인
if firebase_available:
    print("✅ Firebase가 성공적으로 초기화되었습니다.")
    
    # DataIntegrator 초기화
    integrator = DataIntegrator()
    
    # Firebase 데이터 로드 테스트
    print("Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    if not firebase_data.empty:
        print("Firebase 데이터 샘플:")
        print(firebase_data.head())
    else:
        print("⚠️ Firebase에서 데이터를 찾을 수 없습니다.")
else:
    print("❌ Firebase 초기화에 실패했습니다.")
    print("Firebase 설정 파일을 확인해주세요.")


✅ Firebase가 성공적으로 초기화되었습니다.
Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
Firebase 데이터 샘플:
                        user_id movie_id  rating
0  SOMV3U3OEWT7LG9t2VMeNXKQchd2  m5xMKrW     1.0
1  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5x8x5     5.0
2  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5xvEO     3.5
3  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJOV1RW     4.0
4  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mOVgp1d     5.0


In [4]:
# 데이터 통합 테스트
if firebase_available:
    print("=== 데이터 통합 테스트 ===")
    
    # 1. 기존 데이터 로드
    print("1. 기존 데이터를 로드하는 중...")
    original_data = integrator.load_original_data()
    print(f"기존 데이터: {len(original_data)}개 평점")
    
    # 2. Firebase 데이터 로드
    print("2. Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    # 3. 데이터 통합
    if not original_data.empty or not firebase_data.empty:
        print("3. 데이터를 통합하는 중...")
        integrated_data = integrator.integrate_data(original_data, firebase_data)
        print(f"통합된 데이터: {len(integrated_data)}개 평점")
        
        if not integrated_data.empty:
            # 통계 정보
            stats = integrator.get_data_statistics(integrated_data)
            print("\n=== 통합된 데이터 통계 ===")
            print(f"총 평점 수: {stats['total_ratings']:,}")
            print(f"고유 사용자: {stats['unique_users']:,}")
            print(f"고유 영화: {stats['unique_movies']:,}")
            print(f"평균 평점: {stats['avg_rating']:.2f}")
            
            # 데이터 샘플
            print("\n=== 통합된 데이터 샘플 ===")
            print(integrated_data.head())
        else:
            print("⚠️ 통합된 데이터가 비어있습니다.")
    else:
        print("⚠️ 통합할 데이터가 없습니다.")
else:
    print("Firebase가 초기화되지 않아 데이터 통합을 진행할 수 없습니다.")


=== 데이터 통합 테스트 ===
1. 기존 데이터를 로드하는 중...
기존 데이터: 3361101개 평점
2. Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
3. 데이터를 통합하는 중...
통합된 데이터: 3361113개 평점

=== 통합된 데이터 통계 ===
총 평점 수: 3,361,113
고유 사용자: 4,458
고유 영화: 49,856
평균 평점: 3.32

=== 통합된 데이터 샘플 ===
         user_id movie_id   movie_title  rating
0  ZBm5R0Mj7qd46  m5XKVDL         양자물리학     3.0
1  ZBm5R0Mj7qd46  m5Nn2GE       영화로운 작음     3.5
2  ZBm5R0Mj7qd46  mWvq0vZ           내 방     2.5
3  ZBm5R0Mj7qd46  mdKpyBk      성적표의 김민영     3.0
4  ZBm5R0Mj7qd46  mOopbwm  킹덤4: 대장군의 귀환     3.0


In [ ]:
# Firebase 설정 파일 확인
import os
from pathlib import Path

# Firebase 설정 파일 경로들 확인
firebase_config_paths = [
    "movie-recommendation-5bf7f-firebase-adminsdk-fbsvc-9818879c1d.json",
    "../movie-recommendation-5bf7f-firebase-adminsdk-fbsvc-9818879c1d.json",
    "../../movie-recommendation-5bf7f-firebase-adminsdk-fbsvc-9818879c1d.json"
]

print("=== Firebase 설정 파일 확인 ===")
for path in firebase_config_paths:
    if os.path.exists(path):
        print(f"✅ Firebase 설정 파일 발견: {path}")
        break
else:
    print("❌ Firebase 설정 파일을 찾을 수 없습니다.")
    print("다음 경로들을 확인해주세요:")
    for path in firebase_config_paths:
        print(f"  - {path}")

# 현재 작업 디렉토리 확인
print(f"\n현재 작업 디렉토리: {os.getcwd()}")
print(f"프로젝트 루트: {project_root}")

# Firebase 설정 파일이 있는지 확인
firebase_key_file = project_root / "movie-recommendation-5bf7f-firebase-adminsdk-fbsvc-9818879c1d.json"
if firebase_key_file.exists():
    print(f"✅ Firebase 키 파일 발견: {firebase_key_file}")
else:
    print(f"❌ Firebase 키 파일을 찾을 수 없습니다: {firebase_key_file}")


In [18]:
from modeling.utils.data_integration import DataIntegrator


integrator = DataIntegrator()
                    
# 1. 기존 데이터 로드
original_data = integrator.load_original_data()

# 2. Firebase 데이터 로드
firebase_data = integrator.load_firebase_data()

# 3. 데이터 통합
integrated_data = integrator.integrate_data(original_data, firebase_data)

모든 사용자 평점 조회 중 오류: Firebase가 초기화되지 않았습니다.
Firebase에서 데이터를 찾을 수 없습니다.


In [19]:
firebase_data

""
